# Projet TAL M1 S1 : POS Tagging
structure du projet expliqué ici : https://u-paris.zoom.us/rec/share/rFB-gp7vkgMK31kfPxEsyNF2k6q7mHcq3P9MQiovimkr9ebIewZpuD62RBk6SmM.31Mmzx1VyMqQXRtu?startTime=1607605609000

# Implémentation du classifieur

## Import du corpus
Cours Zoom du 17/12 - 35 min

On charge les trois corpus *in-domain* sous la forme de listes de listes de dictionnaires : chaque phrase est une liste de dictionnaire avec une clef : le mot, associée à son POS tag. 

On utilise les trois corpus distincts de French-GSD :
- Train : apprentissage.
- Dev : validation. Pour tester et améliorer le modèle. 
- Test : évaluation. On ne l'utilisera pas pendant l'apprentissage ou le test.

In [1]:
def load_corpus(file):
    with open(file, "r") as f:
        content = f.read() # chargement du corpus
    content = content.split("\n\n") # séparation en phrases
    corpus = []
    for phrase in content: # pour chaque phrase
        phrase_list = [] # liste qui contiendra 1 dictionnaire par mot de la phrase
        for line in phrase.splitlines():
            dico_mot = {} # dictionnaire qui contiendra les features d'un mot (mot, lemme, pos)
            if not line.startswith("#"): # on ignore les lignes qui commencent par #
                features = line.split("\t")
                dico_mot["mot"] = features[1] ####### 
                dico_mot["POS"] = features[3]
                phrase_list.append(dico_mot)
        corpus.append(phrase_list)
    return corpus

gsd_train = load_corpus("corpus-in-domain/fr_gsd-ud-train.conllu")
gsd_test = load_corpus("corpus-in-domain/fr_gsd-ud-test.conllu")
gsd_dev = load_corpus("corpus-in-domain/fr_gsd-ud-dev.conllu")

In [2]:
print("---- Aperçus d'une phrase de chaque corpus-----", end="\n\n")
print(gsd_train[1], end="\n\n")
print(gsd_test[100], end="\n\n")
print(gsd_dev[564])

---- Aperçus d'une phrase de chaque corpus-----

[{'mot': "L'", 'POS': 'DET'}, {'mot': 'œuvre', 'POS': 'NOUN'}, {'mot': 'est', 'POS': 'AUX'}, {'mot': 'située', 'POS': 'VERB'}, {'mot': 'dans', 'POS': 'ADP'}, {'mot': 'la', 'POS': 'DET'}, {'mot': 'galerie', 'POS': 'NOUN'}, {'mot': 'des', 'POS': '_'}, {'mot': 'de', 'POS': 'ADP'}, {'mot': 'les', 'POS': 'DET'}, {'mot': 'batailles', 'POS': 'NOUN'}, {'mot': ',', 'POS': 'PUNCT'}, {'mot': 'dans', 'POS': 'ADP'}, {'mot': 'le', 'POS': 'DET'}, {'mot': 'château', 'POS': 'NOUN'}, {'mot': 'de', 'POS': 'ADP'}, {'mot': 'Versailles', 'POS': 'PROPN'}, {'mot': '.', 'POS': 'PUNCT'}]

[{'mot': 'Ce', 'POS': 'DET'}, {'mot': 'résultat', 'POS': 'NOUN'}, {'mot': 'provient', 'POS': 'VERB'}, {'mot': 'à', 'POS': 'ADP'}, {'mot': 'hauteur', 'POS': 'NOUN'}, {'mot': 'de', 'POS': 'ADP'}, {'mot': '18,5', 'POS': 'NUM'}, {'mot': 'ME', 'POS': 'NOUN'}, {'mot': 'de', 'POS': 'ADP'}, {'mot': 'la', 'POS': 'DET'}, {'mot': 'progression', 'POS': 'NOUN'}, {'mot': 'de', 'POS': 'ADP'}, 

## Extraction des caractéristiques
Pour clarifier le pipeline, nous allons construire de nouveaux dictionnaires qui contiennent pour chaque mot toutes les caractéristiques que nous allons utiliser dans le perceptron :
- mot
- mot précédent
- mot suivant
- commence par une lettre majuscule
- est numérique
- contient des caractères non alphanumériques

In [3]:
def feature_extraction(corpus):
    
    # clés des dictionnaires
    mot = "mot"
    pos = "POS"
    mot_prec = "préc"
    mot_suiv = "suiv"
    maj = "maj"
    num = "nb"
    nonAlphanum = "non alphanum"

    corpus_features = corpus.copy() # copie de la liste pour ne pas modifier la liste d'origine

    for phrase in corpus_features: # ajout des features additionnelles
        for prev, word, suiv in zip([{mot : "START"}] + phrase[:-1], phrase, phrase[1:] + [{mot : "END"}]): # on crée des triplets de mot précédent, mot, mot suivant
            # avec "START" en prev pour le 1er mot
            # et "END" en suiv pour le dernier
            word.update({
                mot_prec : prev[mot],
                mot_suiv : suiv[mot],
                maj : word[mot].istitle(),
                num : word[mot].isnumeric(),
                nonAlphanum : not word[mot].isalnum()})

    return corpus_features

gsd_train_features = feature_extraction(gsd_train)
print(*gsd_train_features[4], sep="\n")

{'mot': 'Ismene', 'POS': 'PROPN', 'préc': 'START', 'suiv': 'entre', 'maj': True, 'nb': False, 'non alphanum': False}
{'mot': 'entre', 'POS': 'VERB', 'préc': 'Ismene', 'suiv': 'et', 'maj': False, 'nb': False, 'non alphanum': False}
{'mot': 'et', 'POS': 'CCONJ', 'préc': 'entre', 'suiv': 'annonce', 'maj': False, 'nb': False, 'non alphanum': False}
{'mot': 'annonce', 'POS': 'VERB', 'préc': 'et', 'suiv': 'que', 'maj': False, 'nb': False, 'non alphanum': False}
{'mot': 'que', 'POS': 'SCONJ', 'préc': 'annonce', 'suiv': "c'", 'maj': False, 'nb': False, 'non alphanum': False}
{'mot': "c'", 'POS': 'PRON', 'préc': 'que', 'suiv': 'est', 'maj': False, 'nb': False, 'non alphanum': True}
{'mot': 'est', 'POS': 'AUX', 'préc': "c'", 'suiv': 'Farnace', 'maj': False, 'nb': False, 'non alphanum': False}
{'mot': 'Farnace', 'POS': 'PROPN', 'préc': 'est', 'suiv': 'qui', 'maj': True, 'nb': False, 'non alphanum': False}
{'mot': 'qui', 'POS': 'PRON', 'préc': 'Farnace', 'suiv': 'a', 'maj': False, 'nb': False, 'no

## Implémentation de l'algorithme de classification
Il faut choisir une régression logistique ou un perceptron moyenné. > plutôt perceptron (explication dans zoom 17/12)

## Evaluation des performances sur le corpus de validation (Dev)


## Evaluation sur le corpus Test

# Evaluation hors-domaine

## Analyse de l'impact du changement de domaine
* identifier causes de la baisse de performance : analyse de sortie, matrice de confusion, erreurs les + fréquentes
* réfléchir à des caractéristiques plus adaptées
* sélection d'un nouvel ensemble d'apprentissage avec des exemples représentatif (généré par un modèle de langue type TP2)

### Corpus UGC (User-generated content)

### Corpus littéraire

## Développement de systèmes robutes au changement de domaine